In [1]:
import pandas as pd
import numpy as np
import os
# import src

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import nltk
# nltk.download()
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

from autocorrect import Speller
from pycontractions import Contractions

from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

In [81]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer 
from text_preprocessing import TextToTensor

ModuleNotFoundError: No module named 'text_preprocessing'

In [5]:
# # Load your favorite word2vec model
# cont = Contractions('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')
# text = "we're"
# text = list(cont.expand_texts([text], precise=True))[0]

In [57]:
def conv_dataframes(pos_path, neg_path):
    with open(pos_path,'r') as f:
        data_p = f.readlines()
    print(data_p[11])
    with open(neg_path,'r') as f:
        data_n = f.readlines()
    pos_data = pd.DataFrame(data_p, columns = ["Doc Text"])
#     pos_data['Sentiment'] = 1
#     pos_data.columns = ["Doc Text", "Sentiment"]
    neg_data = pd.DataFrame(data_n, columns = ["Doc Text"])
#     neg_data['Sentiment'] = -1
#     neg_data.columns = ["Doc Text", "Sentiment"]
    return pos_data, neg_data


## The code for pos tagging and lemmatize sentence is fron the following link:
 ### https://medium.com/@gaurav5430/using-nltk-for-lemmatizing-sentences-c1bfff963258

In [58]:
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [59]:
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
#     print(wordnet_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
#         if tag is None:
#             #if there is no available tag, append the token as is
#             lemmatized_sentence.append(word)
#         else:        
#             #else use the tag to lemmatize the token
#             lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
        if tag is not None:
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag)) 

    return " ".join(lemmatized_sentence)

In [60]:
# print(lemmatizer.lemmatize("I am loving it")) #I am loving it
# print(lemmatizer.lemmatize("loving")) #loving
# print(lemmatizer.lemmatize("loving", "v")) #love
# print(lemmatize_sentence("I am loving it")) #I be love it

In [61]:
def text_preprocessing(data):
    data["Doc Text"] = data["Doc Text"].str.replace('\n\n', ' ')
    data["Doc Text"] = data["Doc Text"].str.replace('[^\w\s]',' ')
    data["Doc Text"] = data["Doc Text"].str.replace('\s+[a-zA-Z]\s+',' ')
    data["Doc Text"] = data["Doc Text"].str.replace('\^[a-zA-Z]\s+',' ')

    data["Doc Text"] = data["Doc Text"].str.replace('\s+',' ')

    # Remove stop words from text
    data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_list]))
    
#     data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([lemmatizer.lemmatize(y) for y in x.split()]))
    data["Doc Text"] = data["Doc Text"].apply(lambda x: lemmatize_sentence(x))

    return data

In [62]:
def create_vocab(pos_data, neg_data):
    temp_pos = text_preprocessing(pos_data)
    temp_neg = text_preprocessing(neg_data)
    temp_pos2 = np.asarray([word_tokenize(re.sub(r"\b[a-zA-Z]\b", " ",i)) for i in temp_pos['Doc Text']])
    temp_neg2 = np.asarray([word_tokenize(re.sub(r"\b[a-zA-Z]\b", " ",i)) for i in temp_neg['Doc Text']]) 
    temp_pos["Doc Text Tokens"] = temp_pos2 
    temp_neg["Doc Text Tokens"] = temp_neg2
    return temp_pos, temp_neg

In [63]:
file_path  = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 594 Deep Learning for NLP\Assignments\rt-polaritydata' 
pos_path = os.path.join(file_path, 'rt-polarity.pos')
neg_path = os.path.join(file_path, 'rt-polarity.neg')

In [64]:
pos_data, neg_data = conv_dataframes(pos_path, neg_path)
pos_data.head()

what really surprises about wisegirls is its low-key quality and genuine tenderness . 



,Doc Text
0,the rock is destined to be the 21st century's ...
1,"the gorgeously elaborate continuation of "" the..."
2,effective but too-tepid biopic\n
3,if you sometimes like to go to the movies to h...
4,"emerges as something rare , an issue movie tha..."


In [65]:
pos_data['Doc Text'][11]

'what really surprises about wisegirls is its low-key quality and genuine tenderness . \n'

In [66]:
# hm_lines = 5331

# tokenizer = nltk.RegexpTokenizer(r"\w+")
# spell = Speller(lang='en')
# # spell = SpellChecker()

# stop_words = set(stopwords.words('english'))
stop_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer() 

vocab_p, vocab_n = create_vocab(pos_data, neg_data)

In [67]:
vocab_p['Doc Text'][11]

'really surprise wisegirls low key quality genuine tenderness'

In [68]:
len(vocab_p)

5331

In [69]:
def pre_trained_vectors():
    embeddings = {}
    with open("glove.6B.300d.txt", 'r', encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings[word] = vector
    return embeddings

In [19]:
embeddings = pre_trained_vectors()

In [20]:
# embeddings['wisegirls']

In [21]:
# np.zeros(300)

In [70]:
def matching_vectors(embeddings, vocab):
    temp1 = []
    mis_word_vector = np.zeros(300)
    k = 0
    mis = 0
    mis_sentence = []
    for i in vocab["Doc Text Tokens"]:
        temp2 =[]
        for j in i:
            try: 
                temp2.append(embeddings[j])
            except:
#                 print(i)
#                 print(j)
#                 print(k)
                mis += 1
                temp2.append(mis_word_vector)
                if i not in mis_sentence:
                    mis_sentence.append(i)
                
#                 print(z)
        k += 1
        temp1.append(np.asarray(temp2))
    return np.asarray(temp1), mis, np.asarray(mis_sentence)
        

In [76]:
def tensorflow_tokens(data):
    tokenizer_tf.fit_on_texts(data["Doc Text"])
    # Creating the padded input for the deep learning model
    max_len = np.max([len(text.split()) for text in data["Doc Text"]])
    TextToTensor_instance = TextToTensor(
        tokenizer=tokenizer, 
        max_len=max_len
        )
    X_train = TextToTensor_instance.string_to_tensor(X_train)

In [77]:
vocab_p.head()

,Doc Text,Doc Text Tokens
0,rock destine 21st century new conan go make sp...,"[rock, destine, 21st, century, new, conan, go,..."
1,gorgeously elaborate continuation lord ring tr...,"[gorgeously, elaborate, continuation, lord, ri..."
2,effective tepid biopic,"[effective, tepid, biopic]"
3,sometimes go movie fun wasabi good place start,"[sometimes, go, movie, fun, wasabi, good, plac..."
4,emerges something rare issue movie honest keen...,"[emerges, something, rare, issue, movie, hones..."


In [78]:
tokenizer_tf  = Tokenizer()


In [79]:
vocab_p_vectors, mis_p, mis_sentence_p = matching_vectors(embeddings, vocab_p)
vocab_p_tf_tokens = tensorflow_tokens(vocab_p)
print(mis_p)
len(mis_sentence_p)

492


348

In [80]:
vocab_p_tf_tokens

In [24]:
#vocab_p_vectors

In [25]:
# mis_sentence_p

In [26]:
vocab_n_vectors, mis_n, mis_sentence_n = matching_vectors(embeddings, vocab_n)
print(mis_n)
len(mis_sentence_n)

415


335

In [27]:
# tf.convert_to_tensor(vocab_n_vectors)

In [28]:
vocab_p_vectors_df = pd.DataFrame(vocab_p_vectors, columns= ['Word Vectors'])
vocab_p_vectors_df['Sentiment'] = 1

vocab_p_vectors_df.head()


,Word Vectors,Sentiment
0,"[[-0.14924, 0.021244, -0.3424, 0.1308, 0.32894...",1
1,"[[-0.49273, 0.051246, 0.030769, -0.22131, -0.6...",1
2,"[[-0.233, -0.14849, 0.48933, -0.29732, 0.57023...",1
3,"[[-0.19219, 0.45566, 0.18116, 0.10643, 0.22674...",1
4,"[[-0.31396, -0.10898, 0.060033, -0.10937, 0.33...",1


In [29]:
vocab_n_vectors_df = pd.DataFrame(vocab_n_vectors, columns= ['Word Vectors'])
vocab_n_vectors_df['Sentiment'] = -1
vocab_n_vectors_df.head()

,Word Vectors,Sentiment
0,"[[-0.19033, 0.035798, -0.058755, 0.022194, -0....",-1
1,"[[0.43384, 0.26893, -0.41571, 0.068764, -0.041...",-1
2,"[[0.050939, 0.28357, 0.18368, 0.097294, -0.549...",-1
3,"[[-0.09981, -0.066583, -1.0622, 0.88628, 0.290...",-1
4,"[[0.11787, 0.32498, -0.21995, -0.43761, -0.123...",-1


## Splitting the data into training, and testing

In [30]:
vocab_p_train1, vocab_p_test= train_test_split(vocab_p_vectors_df, test_size = 800)
vocab_p_train, vocab_p_val = train_test_split(vocab_p_train1, test_size = 800)
vocab_p_test.head()

,Word Vectors,Sentiment
3530,"[[-0.030351, -0.17345, -0.097576, -0.20939, -0...",1
169,"[[-0.13602, -0.11594, -0.017078, -0.29256, 0.0...",1
3100,"[[0.2793, 0.18372, -0.11257, 0.21734, -0.21657...",1
1224,"[[0.089135, 0.71792, 0.028687, 0.21612, 0.0341...",1
4493,"[[-0.40831, -0.058754, -0.0028305, 0.0066421, ...",1


In [31]:
len(vocab_p_val)

800

In [32]:
vocab_n_train1, vocab_n_test= train_test_split(vocab_n_vectors_df, test_size = 800)
vocab_n_train, vocab_n_val = train_test_split(vocab_n_train1, test_size = 800)
vocab_n_test.head()

,Word Vectors,Sentiment
4854,"[[0.00053851, -0.20169, 0.19187, -0.33644, -0....",-1
710,"[[-0.28361, 0.15371, 0.21748, -0.29038, 0.4138...",-1
4881,"[[0.14896, 0.15627, -0.23221, -0.29567, 0.3447...",-1
1987,"[[0.12232, -0.11482, 0.068955, 0.28036, 0.1456...",-1
3344,"[[-0.068696, -0.39391, 0.45735, -0.28542, 0.04...",-1


In [33]:
len(vocab_n_test)

800

In [34]:
training = pd.concat([vocab_p_train,vocab_n_train]).reset_index(drop = True)
validation = pd.concat([vocab_p_val,vocab_n_val]).reset_index(drop = True)
test = pd.concat([vocab_p_test,vocab_n_test]).reset_index(drop = True)
len(validation)

1600

In [35]:
# training.to_csv('Training data.csv')
# validation.to_csv('Validation data.csv')
# test.to_csv('Test data.csv')


In [36]:
len(training['Word Vectors'][5])

6

In [37]:
# tf.convert_to_tensor(np.asarray(training['Word Vectors']).astype(np.float32))

In [38]:
Sequences = np.asarray(training['Word Vectors'])
Targets   = np.asarray( training['Sentiment'])
print(Sequences.shape)

Sequences = np.expand_dims(Sequences, -1)
Targets   = np.expand_dims(Targets, -1)
Sequences.shape

(7462,)


(7462, 1)

## Model building

In [39]:
vocab_size = 5000
embedding_dim = 64
max_length = 7

In [48]:
# training['Word Vectors'][0][0]

In [45]:
# train_padded = pad_sequences(training['Word Vectors'][0][0], maxlen=max_length)
# print(train_padded[[0]])

In [46]:

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='tanh'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


In [47]:
history = model.fit(training['Word Vectors'], training['Sentiment'], epochs=10, batch_size=128, validation_split= None)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [113]:
# c = 0
# data = pd.DataFrame()
# for file in os.listdir(file_path):
#     with open(os.path.join(file_path, file), 'r', encoding='utf-8') as infile:
#         txt = infile.read()
#     data = data.append([[txt, c]], ignore_index=True)
#     c += 1

In [ ]:
# def create_vocab(pos,neg):

#     vocab_p = []
#     vocab_n= []
#     tokenizer = nltk.RegexpTokenizer(r"\w+")
#     with open(pos,'r') as f:
#         contents = f.readlines()
#         for l in contents[:hm_lines]:
#             words = tokenizer.tokenize(l)
#             filtered_sentence = [] 

#             for w in words: 
#                 if w not in stop_words: 
#                     filtered_sentence.append(w) 
# #             print(filtered_sentence)
#             vocab_p.append(list(filtered_sentence))
# #             print(vocab_p)
# #             print(z)

#     with open(neg,'r') as f:
#             words = tokenizer.tokenize(l)
#             filtered_sentence = [] 

#             for w in words: 
#                 if w not in stop_words: 
#                     filtered_sentence.append(w) 
#             vocab_n.append(list(filtered_sentence))
#     return vocab_p, vocab_n

In [ ]:
# def create_vocab(pos,neg):

#     vocab_p = []
#     vocab_n= []
#     with open(pos,'r') as f:
#         contents = f.readlines()
#         for k in contents[:hm_lines]:
#             l = k.replace('\s+[a-zA-Z]\s+','')
#             tokens = tokenizer.tokenize(l)
#             words = [spell(word) for word in tokens if word.isalpha()]
#             filtered_sentence = [] 

#             for w in words: 
#                 if w not in stop_words: 
#                     filtered_sentence.append(w) 
# #             print(filtered_sentence)
#             vocab_p.append(list(filtered_sentence))
# #             print(vocab_p)
# #             print(z)

#     with open(neg,'r') as f:
#             tokens = tokenizer.tokenize(l)
#             words = [spell(word) for word in tokens if word.isalpha()]
#             filtered_sentence = [] 

#             for w in words: 
#                 if w not in stop_words: 
#                     filtered_sentence.append(w) 
#             vocab_n.append(list(filtered_sentence))
#     return np.asarray(vocab_p), np.asarray(vocab_n)

In [153]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dense
max_features = 10000
maxlen = 500
batch_size = 32

print('Loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data( num_words=max_features)
print(input_train[0])
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')
print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Loading data...
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
25000 train sequences
25000 test sequences
P

In [152]:
input_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   